In [15]:
from datasets import load_dataset
from PIL import Image
import os
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import faiss

In [2]:
os.makedirs("tiny_imagenet/train", exist_ok=True)
os.makedirs("tiny_imagenet/val", exist_ok=True)

In [3]:
train_ds = load_dataset('Maysee/tiny-imagenet', split='train')
val_ds = load_dataset('Maysee/tiny-imagenet', split='valid')  

for i, item in enumerate(train_ds):
    img = item['image']
    label = item['label']
    label_folder = f"tiny_imagenet/train/{label}"
    os.makedirs(label_folder, exist_ok=True)
    img.save(f"{label_folder}/{i}.jpg")

for i, item in enumerate(val_ds):
    img = item['image']
    label = item['label']
    label_folder = f"tiny_imagenet/val/{label}"
    os.makedirs(label_folder, exist_ok=True)
    img.save(f"{label_folder}/{i}.jpg")

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
model.eval()

def extract_feature(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
    return outputs[0].cpu().numpy()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [11]:
folders = ["tiny_imagenet/train", "tiny_imagenet/val"]

all_features = []
all_paths = []

for folder in folders:
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if not os.path.isdir(label_path):
            continue

        for filename in os.listdir(label_path):
            img_path = os.path.join(label_path, filename)
            if filename.endswith(".jpg"):
                feature = extract_feature(img_path)
                all_features.append(feature)
                all_paths.append(img_path)

features_array = np.vstack(all_features)
np.savez_compressed("image_features_clip.npz", features=features_array, paths=all_paths)


In [13]:
data = np.load("image_features_clip.npz", allow_pickle=True)

In [14]:
features = data["features"].astype("float32")  
paths = data["paths"]

faiss.normalize_L2(features)
index = faiss.IndexFlatIP(features.shape[1])
index.add(features)

faiss.write_index(index, "clip_faiss_index.index")
np.save("clip_image_paths.npy", paths)